# ML Learning Bytes Agent

This notebook demonstrates how to create an AI agent system that generates and sends educational content about Machine Learning via email. The system consists of multiple specialized agents working together to create learning materials.

## Overview
- The system generates three types of ML learning content:
  1. Core ML concepts with explanations and examples
  2. ML interview questions and answers
  3. Small ML coding problems
- Content is formatted as HTML emails and sent to users
- Uses SendGrid for email delivery

## System Architecture
1. **Specialized Agents**:
   - ML Concept Agent: Explains core ML concepts
   - Interview Q&A Agent: Creates interview questions and answers
   - Mini Problem Agent: Generates coding problems
   - HTML Converter Agent: Formats content for email

2. **Learning Manager**: Coordinates the entire system by:
   - Receiving user requests
   - Selecting appropriate specialized agent
   - Formatting content
   - Sending emails
   

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [25]:
# Import required libraries
from dotenv import load_dotenv  # For loading environment variables
from agents import Agent, Runner, trace, function_tool  # Custom agent framework
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid  # For email functionality
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio  # For async operations

In [26]:
load_dotenv(override=True)

True

## Step 1: Creating instructions for each agent

In [27]:
# Instructions for the ML Concept Agent - generates explanations of ML concepts

instructions1 = """Explain a core machine learning concept in under 150 words. Include:

A simple definition

An intuitive explanation (preferably with analogy)

A real-world use case

A small code snippet if relevant
Output should be in html format suitable for email body
Startiing should be introduction to mail stating that it is a learning bite"""

# Instructions for the ML Interview Q&A Agent - generates interview questions and answers
instructions2 = """Pose a realistic scenario-based ML interview question and provide:

The question

A structured reasoning process (e.g., steps or options)

A concise sample answer (2–4 bullet points)

A bonus tip or common mistake to avoid
Output should be in html format suitable for email body
Startiing should be introduction to mail stating that it is a ML interview question"""

# Instructions for the Mini Problem Agent - generates small ML coding problems
instructions3 = """Create a small ML coding problem that can be solved in under 30 minutes. Include:

A brief scenario

Clear input/output expectations

Starter code and a hint
Output should be in html format suitable for email body
Startiing should be introduction to mail stating that it is a ML coding problem"""

# Instructions for the HTML Converter Agent - converts text to HTML
html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

## Steps 2: Creating agents


In [28]:
# Creating the ML Concept Agent
learning_agent1 = Agent(
        name="ML Concept Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

# Creating the ML Interview Q&A Agent
learning_agent2 = Agent(
        name="Interview Q&A Agent ",
        instructions=instructions2,
        model="gpt-4o-mini"
)

# Creating the Mini Problem Agent
learning_agent3 = Agent(
        name="Mini Problem Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

# Creating the HTML Converter Agent
html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")


## Steps 3: Creating tools for above agents

Simply wrap your function with the decorator `@function_tool`

In [29]:
# Tool for sending HTML emails using SendGrid
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('learning_agent_key'))
    from_email = Email("sandyyy211@gmail.com")  # Change to your verified sender
    to_email = To("sandyyy11@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, "ML Learning Bite", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

### And you can also convert an Agent into a tool



# A tool for each of our 4 email-writing agents

# And a tool for our function to send emails

In [30]:

# Convert each agent into a tool with specific names and descriptions
# ML Concept Agent tool for explaining core ML concepts
tool1 = learning_agent1.as_tool(tool_name="learning_agent1", tool_description=instructions1)

# Interview Q&A Agent tool for generating interview questions and answers
tool2 = learning_agent2.as_tool(tool_name="learning_agent2", tool_description=instructions2)

# Mini Problem Agent tool for creating coding problems
tool3 = learning_agent3.as_tool(tool_name="learning_agent3", tool_description=instructions3)

# HTML Converter tool for formatting content into email-friendly HTML
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

# Combine all tools into a list for the learning manager to use
tools = [tool1, tool2, tool3, html_tool, send_html_email]

# Display the list of available tools
tools

[FunctionTool(name='learning_agent1', description='Explain a core machine learning concept in under 150 words. Include:\n\nA simple definition\n\nAn intuitive explanation (preferably with analogy)\n\nA real-world use case\n\nA small code snippet if relevant\nOutput should be in html format suitable for email body\nStartiing should be introduction to mail stating that it is a learning bite', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'learning_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10aabe020>, strict_json_schema=True),
 FunctionTool(name='learning_agent2', description='Pose a realistic scenario-based ML interview question and provide:\n\nThe question\n\nA structured reasoning process (e.g., steps or options)\n\nA concise sample answer (2–4 bullet points)\n\nA bonus tip or common mistake to avo

## And now it's time for our Learning Manager - our planning agent that will autonomously pick up right agent

In [31]:
# Define instructions for the Learning Manager agent
# The agent is responsible for coordinating ML learning content generation and delivery
instructions ="""You are a ML learning resource agent. You use the tools given to you to generate emails having learning bites for ML. \
You never generate learning bites emails yourself; you always use the tools. \
You have 3 learning_agent tools at your disposal. \
Based on user input you pick the relevant agent and generate content.
Then use html_converter tool to convert the content to html format.
Then use the send_email tool to send the  email to the user."""

# Initialize the Learning Manager agent with specified name, instructions, tools and model
learning_manager = Agent(name="Learning Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

# Define the user's query about regularization in ML
message = "Explain regularisation in Machine Learning"

# Execute the learning manager with tracing enabled to monitor the process
with trace("Learning manager"):
    result = await Runner.run(learning_manager, message)

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!
